In [1]:
import json
from pathlib import Path
from typing import Any, Dict, Literal, Optional

import cv2
import numpy as np

from nerfstudio.data.utils.colmap_parsing_utils import (
    qvec2rotmat,
    read_cameras_binary,
    read_images_binary,
    read_points3D_binary,
)
from nerfstudio.process_data.process_data_utils import CameraModel


In [2]:
def parse_colmap_camera_params(camera) -> Dict[str, Any]:  # pylint: disable=too-many-statements
    """
    Parses all currently supported COLMAP cameras into the transforms.json metadata

    Args:
        camera: COLMAP camera
    Returns:
        transforms.json metadata containing camera's intrinsics and distortion parameters

    """
    out: Dict[str, Any] = {
        "w": camera.width,
        "h": camera.height,
    }

    # Parameters match https://github.com/colmap/colmap/blob/dev/src/base/camera_models.h
    camera_params = camera.params
    if camera.model == "SIMPLE_PINHOLE":
        # du = 0
        # dv = 0
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[0])
        out["cx"] = float(camera_params[1])
        out["cy"] = float(camera_params[2])
        out["k1"] = 0.0
        out["k2"] = 0.0
        out["p1"] = 0.0
        out["p2"] = 0.0
        camera_model = CameraModel.OPENCV
    elif camera.model == "PINHOLE":
        # f, cx, cy, k

        # du = 0
        # dv = 0
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[1])
        out["cx"] = float(camera_params[2])
        out["cy"] = float(camera_params[3])
        out["k1"] = 0.0
        out["k2"] = 0.0
        out["p1"] = 0.0
        out["p2"] = 0.0
        camera_model = CameraModel.OPENCV
    elif camera.model == "SIMPLE_RADIAL":
        # f, cx, cy, k

        # r2 = u**2 + v**2;
        # radial = k * r2
        # du = u * radial
        # dv = u * radial
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[0])
        out["cx"] = float(camera_params[1])
        out["cy"] = float(camera_params[2])
        out["k1"] = float(camera_params[3])
        out["k2"] = 0.0
        out["p1"] = 0.0
        out["p2"] = 0.0
        camera_model = CameraModel.OPENCV
    elif camera.model == "RADIAL":
        # f, cx, cy, k1, k2

        # r2 = u**2 + v**2;
        # radial = k1 * r2 + k2 * r2 ** 2
        # du = u * radial
        # dv = v * radial
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[0])
        out["cx"] = float(camera_params[1])
        out["cy"] = float(camera_params[2])
        out["k1"] = float(camera_params[3])
        out["k2"] = float(camera_params[4])
        out["p1"] = 0.0
        out["p2"] = 0.0
        camera_model = CameraModel.OPENCV
    elif camera.model == "OPENCV":
        # fx, fy, cx, cy, k1, k2, p1, p2

        # uv = u * v;
        # r2 = u**2 + v**2
        # radial = k1 * r2 + k2 * r2 ** 2
        # du = u * radial + 2 * p1 * u*v + p2 * (r2 + 2 * u**2)
        # dv = v * radial + 2 * p2 * u*v + p1 * (r2 + 2 * v**2)
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[1])
        out["cx"] = float(camera_params[2])
        out["cy"] = float(camera_params[3])
        out["k1"] = float(camera_params[4])
        out["k2"] = float(camera_params[5])
        out["p1"] = float(camera_params[6])
        out["p2"] = float(camera_params[7])
        camera_model = CameraModel.OPENCV
    elif camera.model == "OPENCV_FISHEYE":
        # fx, fy, cx, cy, k1, k2, k3, k4

        # r = sqrt(u**2 + v**2)

        # if r > eps:
        #    theta = atan(r)
        #    theta2 = theta ** 2
        #    theta4 = theta2 ** 2
        #    theta6 = theta4 * theta2
        #    theta8 = theta4 ** 2
        #    thetad = theta * (1 + k1 * theta2 + k2 * theta4 + k3 * theta6 + k4 * theta8)
        #    du = u * thetad / r - u;
        #    dv = v * thetad / r - v;
        # else:
        #    du = dv = 0
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[1])
        out["cx"] = float(camera_params[2])
        out["cy"] = float(camera_params[3])
        out["k1"] = float(camera_params[4])
        out["k2"] = float(camera_params[5])
        out["k3"] = float(camera_params[6])
        out["k4"] = float(camera_params[7])
        camera_model = CameraModel.OPENCV_FISHEYE
    elif camera.model == "FULL_OPENCV":
        # fx, fy, cx, cy, k1, k2, p1, p2, k3, k4, k5, k6

        # u2 = u ** 2
        # uv = u * v
        # v2 = v ** 2
        # r2 = u2 + v2
        # r4 = r2 * r2
        # r6 = r4 * r2
        # radial = (1 + k1 * r2 + k2 * r4 + k3 * r6) /
        #          (1 + k4 * r2 + k5 * r4 + k6 * r6)
        # du = u * radial + 2 * p1 * uv + p2 * (r2 + 2 * u2) - u
        # dv = v * radial + 2 * p2 * uv + p1 * (r2 + 2 * v2) - v
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[1])
        out["cx"] = float(camera_params[2])
        out["cy"] = float(camera_params[3])
        out["k1"] = float(camera_params[4])
        out["k2"] = float(camera_params[5])
        out["p1"] = float(camera_params[6])
        out["p2"] = float(camera_params[7])
        out["k3"] = float(camera_params[8])
        out["k4"] = float(camera_params[9])
        out["k5"] = float(camera_params[10])
        out["k6"] = float(camera_params[11])
        raise NotImplementedError(f"{camera.model} camera model is not supported yet!")
    elif camera.model == "FOV":
        # fx, fy, cx, cy, omega
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[1])
        out["cx"] = float(camera_params[2])
        out["cy"] = float(camera_params[3])
        out["omega"] = float(camera_params[4])
        raise NotImplementedError(f"{camera.model} camera model is not supported yet!")
    elif camera.model == "SIMPLE_RADIAL_FISHEYE":
        # f, cx, cy, k

        # r = sqrt(u ** 2 + v ** 2)
        # if r > eps:
        #     theta = atan(r)
        #     theta2 = theta ** 2
        #     thetad = theta * (1 + k * theta2)
        #     du = u * thetad / r - u;
        #     dv = v * thetad / r - v;
        # else:
        #     du = dv = 0
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[0])
        out["cx"] = float(camera_params[1])
        out["cy"] = float(camera_params[2])
        out["k1"] = float(camera_params[3])
        out["k2"] = 0.0
        out["k3"] = 0.0
        out["k4"] = 0.0
        camera_model = CameraModel.OPENCV_FISHEYE
    elif camera.model == "RADIAL_FISHEYE":
        # f, cx, cy, k1, k2

        # r = sqrt(u ** 2 + v ** 2)
        # if r > eps:
        #     theta = atan(r)
        #     theta2 = theta ** 2
        #     theta4 = theta2 ** 2
        #     thetad = theta * (1 + k * theta2)
        #     thetad = theta * (1 + k1 * theta2 + k2 * theta4)
        #     du = u * thetad / r - u;
        #     dv = v * thetad / r - v;
        # else:
        #     du = dv = 0
        out["fl_x"] = float(camera_params[0])
        out["fl_y"] = float(camera_params[0])
        out["cx"] = float(camera_params[1])
        out["cy"] = float(camera_params[2])
        out["k1"] = float(camera_params[3])
        out["k2"] = float(camera_params[4])
        out["k3"] = 0
        out["k4"] = 0
        camera_model = CameraModel.OPENCV_FISHEYE
    else:
        # THIN_PRISM_FISHEYE not supported!
        raise NotImplementedError(f"{camera.model} camera model is not supported yet!")

    out["camera_model"] = camera_model.value
    return out

In [3]:
# path to colmap binary files
dir = Path('/home/nikola/projects/pxo/data/prepared/nerfstudio/living_room/colmap/sparse/0')

## Rewriting `load_colmap_data()` form poses_utils.py

In [4]:
# Read images binary
im_id_to_image = read_images_binary(dir / "images.bin")

#print(im_id_to_image)

# initializes an empty list, which will be used to store world-to-camera matrices
w2c_matrices = []
# initializes an empty list to store image filenames
im_names = []

# for each image id(key) and image data(value) in im_id_to_image
for im_id, im_data in im_id_to_image.items():
    # convert quaternion to rotation matrix
    rotation = qvec2rotmat(im_data.qvec)
    # get translation and reshape it as 3x1 or column vector
    translation = im_data.tvec.reshape(3, 1)
    # concatenate rotation matrix(3x3) and translation matrix(3x1) into a 3x4 matrix
    w2c = np.concatenate([rotation, translation], 1)
    # additional row `[0, 0, 0, 1]` is appended to the bottom of the `w2c` matrix to account
    # for homogeneous coordinates. This row allows for the translation component to be included
    # in the transformation.
    w2c = np.concatenate([w2c, np.array([[0, 0, 0, 1]])], 0)
    # append world-to-camera matrix to w2c_mats list
    w2c_matrices.append(w2c)
    # append image filename to im_names list
    im_names.append(im_data.name)

# get sorted inidices of image filenames that would sort the array
im_names_sorted_indicies = np.argsort(im_names)    

# converts w2c_mats list into a numpy array by stacking the matrices along axis 0 
# Output shape is (606, 4, 4), 606 world-to-camera transformation matrices, 
# where each matrix is of size 4x4. 
w2c_matrices = np.stack(w2c_matrices, 0)
print(f'w2c_matrices.shape: {w2c_matrices.shape}')
print(f'w2c_matrices[0]: \n{w2c_matrices[0]}')

# calculates the inverse of each world-to-camera matrix and stores it in camera-to-world 
# matrixs of shape (606, 4, 4)
c2w_matrices = np.linalg.inv(w2c_matrices)
print(f'c2w_matrices.shape: {c2w_matrices.shape}')
print(f'c2w_matrices[0]: \n{c2w_matrices[0]}')

# `:` include all 606 matrices, 
# `:3` include the first three rows (representing rotation components)
# `:4` include the first four columns (representing translation components) 
# The resulting array will have the shape (606, 3, 4) or (num_matrices, 3, 4), 
# where the first dimension corresponds to the number of transformation matrices, 
# the second dimension represents the rotation components (3 rows), 
# and the third dimension represents the translation omponents (4 columns)
c2w_matrices_extract_R_T = c2w_matrices[:, :3, :4]
print(f'c2w_matrices_extract_R_T.shape: {c2w_matrices_extract_R_T.shape}')  
print(f'c2w_matrices_extract_R_T[0]: \n{c2w_matrices_extract_R_T[0]}')  

# rearanges original array of shape (num_matrices, 3, 4) to (3, 4, num_matrices) or
# (606, 3, 4) to (3, 4, 606)
c2w_matrices_extract_R_T = c2w_matrices_extract_R_T.transpose(1, 2, 0)
print(f'c2w_matrices_extract_R_T.shape:{c2w_matrices_extract_R_T.shape}')
print(f'c2w_matrices_extract_R_T[:, :, 0]: \n{c2w_matrices_extract_R_T[:, :, 0]}')

# If we assume that:  
# 
# `r` - represents the rightward direction or the X-axis,  
# `u` - represents the upward direction or the Y-axis,  
# `t` - represents the direction from the camera towards the scene or the Z-axis,
# 
# which is left-handed coordinate system, then OpenCV coordiante system can be expressed as `[r, -u, t]`  
# and OpenGL coordiante system can be expressed as `[r, u, -t]`.

# because Colmap is using OpenCV coordinate system, our camera matrix is expressed as `[r, -u, t]`
# must switch to [-u, r, -t] from [r, -u, t], NOT OpenGL coordiante system [r, u, -t]
# TODO: what is [-u, r, -t]? Which space? Should it be OpenGL space?
c2w_matrices_swaped = np.concatenate([
                    c2w_matrices_extract_R_T[:, 1:2, :], 
                    c2w_matrices_extract_R_T[:, 0:1, :], 
                    -c2w_matrices_extract_R_T[:, 2:3, :], 
                    c2w_matrices_extract_R_T[:, 3:4, :]
                    ], 1)
print(f'c2w_matrices_swaped.shape:{c2w_matrices_swaped.shape}')
print(f'c2w_matrices_swaped[:, :, 0]: \n{c2w_matrices_swaped[:, :, 0]}')


w2c_matrices.shape: (606, 4, 4)
w2c_matrices[0]: 
[[ 0.87490473  0.11751033 -0.46982235  1.1276482 ]
 [-0.10608506  0.99305713  0.05082803  0.0638054 ]
 [ 0.47253325  0.00537144  0.88129647  0.50718659]
 [ 0.          0.          0.          1.        ]]
c2w_matrices.shape: (606, 4, 4)
c2w_matrices[0]: 
[[ 0.87490473 -0.10608506  0.47253325 -1.21947847]
 [ 0.11751033  0.99305713  0.00537144 -0.19859704]
 [-0.46982235  0.05082803  0.88129647  0.07956947]
 [ 0.          0.          0.          1.        ]]
c2w_matrices_extract_R_T.shape: (606, 3, 4)
c2w_matrices_extract_R_T[0]: 
[[ 0.87490473 -0.10608506  0.47253325 -1.21947847]
 [ 0.11751033  0.99305713  0.00537144 -0.19859704]
 [-0.46982235  0.05082803  0.88129647  0.07956947]]
c2w_matrices_extract_R_T.shape:(3, 4, 606)
c2w_matrices_extract_R_T[:, :, 0]: 
[[ 0.87490473 -0.10608506  0.47253325 -1.21947847]
 [ 0.11751033  0.99305713  0.00537144 -0.19859704]
 [-0.46982235  0.05082803  0.88129647  0.07956947]]
c2w_matrices_swaped.shape:(3,

In [5]:
# Read camera binary
cam_id_to_camera = read_cameras_binary(dir / "cameras.bin")

# if camera id is not 1, raise error, this exception indicates that the code expects 
# only one camera ID to be present in the dictionary, and if there are multiple camera IDs, it cannot proceed
if set(cam_id_to_camera.keys()) != {1}:
    raise RuntimeError("Only single camera shared for all images is supported.")
cam = parse_colmap_camera_params(cam_id_to_camera[1])
print(f'cam: {cam}')  

# get camera height, width, focal length all in pixel values
cam_h, cam_w, cam_f = cam["h"], cam["w"], cam["fl_x"]
# convert tuple to numpy array dimension (3, 1) aka column vector
cam_h_w_f = np.array([cam_h, cam_w, cam_f]).reshape(3, 1)

# extend cam_h_w_f array by adding a new axis at the end
# ... is used as shorthand to include all existing axes in the array
# np.newaxis, a new axis is added at the end of the array resulting in a (3, 1, 1) array
cam_h_w_f_extended = cam_h_w_f[..., np.newaxis]

# tile the cam_h_w_f_extended array (3, 1, 1) to get the shape (3, 1, 606) so it matches the shape 
# of c2w_matrices_extract_R_T (3, 4, 606)
cam_h_w_f_tiled = np.tile(cam_h_w_f_extended, (1, 1, c2w_matrices_extract_R_T.shape[-1]))

# concatenate c2w_matrices_swaped (3, 4, 606) and cam_h_w_f_tiled (3, 1, 606) to get (3, 5, 606)
poses = np.concatenate([c2w_matrices_swaped, cam_h_w_f_tiled], 1)
print(f'poses.shape: {poses.shape}')
with np.printoptions(suppress=True):
    print(f'poses[:, :, 0]: \n{poses[:, :, 0]}')
    

cam: {'w': 1920, 'h': 1080, 'fl_x': 876.0743282427004, 'fl_y': 876.9458371681479, 'cx': 959.3438718548749, 'cy': 541.6632566160417, 'k1': 0.03751194987446759, 'k2': -0.033626984012414166, 'p1': 0.00044331369627067624, 'p2': 0.0003175956738498672, 'camera_model': 'OPENCV'}
poses.shape: (3, 5, 606)
poses[:, :, 0]: 
[[  -0.10608506    0.87490473   -0.47253325   -1.21947847 1080.        ]
 [   0.99305713    0.11751033   -0.00537144   -0.19859704 1920.        ]
 [   0.05082803   -0.46982235   -0.88129647    0.07956947  876.07432824]]


## Rewriting `save_poses()` form poses_utils.py

In [6]:
# Read points3D binary
ptid_to_info = read_points3D_binary(dir / "points3D.bin")

## Miscellaneous 

In [7]:
# comparison between llff camera to world and ns camera to world
llff_c2w = np.eye(4)
llff_c2w = np.concatenate([
                    llff_c2w[:, 1:2], 
                    llff_c2w[:, 0:1], 
                    -llff_c2w[:, 2:3], 
                    llff_c2w[:, 3:4]
                    ], 1)

ns_c2w = np.eye(4)
ns_c2w[0:3, 1:3] *= -1
ns_c2w = ns_c2w[np.array([1, 0, 2, 3]), :]
ns_c2w[2, :] *= -1

print(f'llff_c2w: \n{llff_c2w}')
print(f'ns_c2w: \n{ns_c2w}')

llff_c2w: 
[[ 0.  1. -0.  0.]
 [ 1.  0. -0.  0.]
 [ 0.  0. -1.  0.]
 [ 0.  0. -0.  1.]]
ns_c2w: 
[[ 0. -1. -0.  0.]
 [ 1. -0. -0.  0.]
 [-0.  0.  1. -0.]
 [ 0.  0.  0.  1.]]
